In [25]:
import requests
import pandas as pd
import json 
from dateutil import parser

In [26]:
API_KEY= "7203735f4489a07b1fdaa82e0825b643-037a13c8ecd863561ee2fb38166bda77"
ACCOUNT_ID = "101-011-28651499-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [27]:
session = requests.Session()

In [28]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [29]:
params = dict(
    count = 10,
    granularity="H1",
    price="MBA"
)

In [30]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [31]:
response =session.get(url,params=None, data=None,headers=None)

In [32]:
response.status_code


200

In [33]:
data = response.json()

In [34]:
instruments_list = data['instruments']

In [35]:
len(instruments_list)

127

In [36]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [37]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision','marginRate']

In [38]:
instruments_dict = {}

In [39]:
for i in instruments_list: 
    key = i['name']
    instruments_dict[key] = { k: i[k] for k in key_i}

In [40]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.0333'}

In [41]:
with open ("../data/instruments.json","w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [45]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity=granularity,
        price="MBA"
        )
    response =session.get(url,params=params, data=None,headers=None)
    data = response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
        
    prices = ['mid','bid','ask']
    ohlc = ['o','h','l','c']   
    
    final_data = []
    for candle in data:
        if candle['complete']== False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"]= float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df
def create_data_file(pair_name, count=10, granularity="H1"):
    code,data = fetch_candles(pair_name,count,granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) ==0:
        print("No candles",pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")

In [46]:
code, data = fetch_candles("EUR_USD", count=10,granularity="H4")
candles_df = get_candles_df(data)

In [47]:
create_data_file("EUR_USD", count=10,granularity="H4")

EUR_USD H4 9 candles, 2024-06-25 13:00:00+00:00 2024-06-26 21:00:00+00:00


In [48]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [49]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1","H2"]:
                create_data_file(pr, count=4001, granularity=g)
                

EUR_USD H1 4000 candles, 2023-11-03 11:00:00+00:00 2024-06-27 04:00:00+00:00
EUR_USD H2 4000 candles, 2023-03-15 19:00:00+00:00 2024-06-27 03:00:00+00:00
EUR_GBP H1 4000 candles, 2023-11-03 11:00:00+00:00 2024-06-27 04:00:00+00:00
EUR_GBP H2 4000 candles, 2023-03-15 19:00:00+00:00 2024-06-27 03:00:00+00:00
EUR_JPY H1 4000 candles, 2023-11-03 11:00:00+00:00 2024-06-27 04:00:00+00:00
EUR_JPY H2 4000 candles, 2023-03-15 19:00:00+00:00 2024-06-27 03:00:00+00:00
EUR_CHF H1 4000 candles, 2023-11-03 11:00:00+00:00 2024-06-27 04:00:00+00:00
EUR_CHF H2 4000 candles, 2023-03-15 19:00:00+00:00 2024-06-27 03:00:00+00:00
EUR_NZD H1 4000 candles, 2023-11-03 13:00:00+00:00 2024-06-27 04:00:00+00:00
EUR_NZD H2 4000 candles, 2023-03-15 23:00:00+00:00 2024-06-27 03:00:00+00:00
EUR_CAD H1 4000 candles, 2023-11-03 11:00:00+00:00 2024-06-27 04:00:00+00:00
EUR_CAD H2 4000 candles, 2023-03-15 19:00:00+00:00 2024-06-27 03:00:00+00:00
EUR_AUD H1 4000 candles, 2023-11-03 11:00:00+00:00 2024-06-27 04:00:00+00:00